# Computational Theory Tasks

<div style="background-color:#003366; color:white; padding:8px; border-radius:5px; font-size:14px">
    
#### **Tests**  
**At the end of each task, tests are available to verify functionality.**
</div>


### Imports

In [2]:
# Imports here

## Task 1 - Binary Representations

1.1 Rotate the bits in a 32-bit unsigned integer to the left by `n` places

In [3]:
def rotl(x, n):
    # Get the modulo 32 to avoid unnecessary rotations
    n %= 32

    # Perform the rotation
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

1.2 Rotate the bits in a 32-bit unsigned integer to the right by `n` places

In [4]:
def rotr(x, n):
    # Get the modulo 32 to avoid unnecessary rotations
    n %= 32

    # Perform the right rotation
    return ((x >> n) & 0xFFFFFFFF) | (x << (32 - n))

1.3 Choose the bits from `y` where `x` has bits set to `1` and bits in `z` where `x` has bits set to `0`

In [10]:
def ch(x, y, z):
    return (y & x) | (z & ~x)

1.4 Take a majority vote of the bits in `x`, `y`, and `z`.

In [12]:
def maj(x, y, z):
    return (x & y) | (z & x) | (z & y)

<div style="background-color:#003366; color:white; padding:8px; border-radius:5px; font-size:14px">
    
##### **Test 1A - "Verifying `rotl`"**  
**We will now test `rotl(x, n)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `8`   | `5120` |
| `2153` `8` | `551168` |
</div>


In [6]:
print(rotl(20, 8))
print(rotl(2153, 8))

5120
551168


<div style="background-color:#003366; color:white; padding:8px; border-radius:5px; font-size:14px">
    
##### **Test 1B - "Verifying `rotr`"**  
**We will now test `rotr(x, n)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `8`   | `335544320` |
| `2153` `8` | `36121346056` |
</div>


In [7]:
print(rotr(20, 8))
print(rotr(2153, 8))

335544320
36121346056


<div style="background-color:#003366; color:white; padding:8px; border-radius:5px; font-size:14px">
    
##### **Test 1C - "Verifying `ch`"**  
**We will now test `ch(x, y, z)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `2153` `54`   | `34` |
</div>


In [11]:
ch(20, 2153, 54)

34

<div style="background-color:#003366; color:white; padding:8px; border-radius:5px; font-size:14px">
    
##### **Test 1D - "Verifying `maj`"**  
**We will now test `maj(x, y, z)`**

**Expected results:**

| Input  | Output |
|--------|--------|
| `20` `2153` `54`   | `34` |
</div>


In [ ]:
maj(20, 2153, 54)

### Task 2: Hash Functions


### Task 3: SHA256


### Task 4: Prime Numbers

### Task 5: Roots

### Task 6: Proof of Work

### Task 7: Turing Machines

### Task 8: Computational Complexity